<a href="https://colab.research.google.com/github/Nozasp/PredictiveCoding/blob/main/WongModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
from functools import partial
import numpy as np
import scipy.integrate
import scipy
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches #used to write custom legends
#from param import *
#from param2 import *
import random
#### MESSAGE !

# Parameters

In [ ]:

# ================== Parameters ===============================
# ¤ General parameters
N = 20
#T = 0.2:0.01:1; # periods of the generative oscillator (seconds)
#f = numpy.matrix(range(1, N))   #; # unitless
f = list(range(1, N+1))
dt = 0.001; # seconds
initTime = 1;   # initalisation time (seconds)

# ¤ Runtime
verb  = 'false'
class storeFields:
    def __init__(self,v,dv,x):
        self.v  = v
        self.dv = dv
        self.x = x

storeFields_Init = storeFields(['he', 'hi', 'hs'], [], [])
print(storeFields_Init.v[1], storeFields_Init.dv)



# ¤ time constants
#ostoj   = flse;
taue    = 0.020; # seconds [Brunel 2001] (4.5 +- 2.4 ms)
taui    = 0.010; # seconds [Brunel 2001] (4.5 +- 2.4 ms)
tauNMDA = 0.100; # NMDA-gating time constants (s) [Brunel 2001]
tauGABA = 0.005; # GABA-gating time constants (s) [Brunel 2001]
tauAMPA = 0.002; # AMPA-gating time constants (s) [Brunel 2001]


# ¤ population parameters
gamma = 0.641;  # NMDA coupling   [Brunel 2001]
sigma = 0.0007; # noise amplitude [Wong 2006]   # it s 0.007 (nA) instead !!!
I0e   = 0.2346; # constant population input, exc (nA) [Wong2006]
I0i   = 0.17;   # constant population input, inh (nA)

# ¤ ???? parameter of the phi function
ae = 18.26;
be = -5.38;
hme = 78.67;
ai = 21.97;
bi = -4.81;
hmi = 125.62;



# ¤ conductivities: the strength of the synaptic connection
Jee = 0.2;
Jie = 0.2;
Jei = 1.4;
Jii = 6.7;
Jin = 0.008;
Jiq = 0.85;
Jes = 3.5;
Jsi = 0.12;
Jem = 2.2;

Jext = 0.01 #% nA/Hz; # B_wong easy

# ¤ Input param3ters
In0 = 0;    # Spontaneous firing rate of input populations (Hz)
InMax = 50;   # Max firing rate of input populations (Hz)
Iq0 = 0;    ## Spontaneous firing rate of feedback populations (Hz)
IqMax = 10;   ## Max firing rate of feedback populations (Hz)

c_dash = 90 # imput by default
mu0 = 30 # wong 2006
#External input = Firing rate of upstream neurons
In = Jext * mu0 * (1 - c_dash/100) #Jext average synaptic coupling with AMPARs
Iq = Jext * mu0 * (1 - c_dash/100)
# Ioe or Ioi = input weights I n and Iq??
eta = np.random.normal(size=(4,1))#return N by 4 matrix of normal random number
I_noise = sigma * eta
#I need to add the noise from the external input. I put just I_noise but not the gaussian filter neither other type of noise



# ¤ adaptation dynamics
#alpha    = 0; # adaptation strenght
alpha = 0.022; # adaptation strenght
tauAdapt = 1.50;  # Adaptation time constant (s)



# ¤ connectivity matrices
sigmaIn  = 3;
sigmaEI  = sigmaIn;
sigmaQie = sigmaIn;
sigmaInh = [0.2, sigmaIn];

#. For the plasticity :
#wei = gaussianFilter(sigmaEI, N);
#wes = numpy.identity(N) # eye = identiy matrix of size N*N



hi []


Simulation parameters

In [ ]:
T = 2500 # % ms
dt = 1 
time = list(range(dt,T))
timeIX = list(range(1, len(time)))

Initialisation of the variables

In [29]:
# Initialisation of the variables
r_E = np.zeros((2,len(time)))
r_I = np.zeros((1,len(time)))
r_S = np.zeros((1,len(time))) #NMDA

drE_dt= np.zeros((2,len(time)))
drI_dt= np.zeros((1,len(time)))

S_GABA = np.zeros((1,len(time)))
S_AMDA = np.zeros((2,len(time)))
S_NMDA = np.zeros((1,len(time)))

I_syn = np.zeros((4, len(time)))
Phi_I = np.zeros((4, len(time)))

In [32]:
drE_dt[0][3]

0.0

# Function
function Phi => input output function

In [58]:
def Phi_I(I_syn, typ) :
    a = globals()[''.join(['a', typ])]
    b = globals()[''.join(['b', typ])]
    hm = globals()[''.join(['hm', typ])]
    px = hm * (1 / (1 + math.exp(- (a * I_syn + b))))  
    #px = hm * (1 / (1 + np.exp(- (a * I_syn + b))))  
    #px = hme * (1 / (1 + math.exp(- (ae * I_syn + be))))
    return px

In [109]:
dsGaba_dt = (- S_GABA[0, i-1]/tauGABA) + drI_dt[0][i] * dt + I_noise[1]
dsGaba_dt

array([0.1018602])

In [113]:
max(0,S_GABA[0,0]) + dsGaba_dt[0] *dt

0.00010186020474696374

# Simulation

In [114]:
for i in range(1, len(time)):
    ## / total Synaptic input
    I_syn_he = S_AMDA[0, i -1] * Jee - Jie * I_noise[0,] * S_GABA[0,i-1] + Jin * In0 #Jie * wie * S_GABA
    I_syn_hm = S_AMDA[0, i -1 ] * Jee
    I_syn_hs = Jes * (I_noise[1,] ) * S_AMDA[0, i-1] + Jiq * (Iq0 + S_AMDA[1,i-1])  #(wei * r_S) * S_AMDA[0,]
    I_syn_hi = Jei * (I_noise[2,] ) * S_AMDA[0, i-1] - Jii * I_noise[3,] * S_GABA[0,i-1] + Jsi * S_NMDA[0,i-1] #Jii * wii * r_I * S_GABA
    Phi_he = Phi_I(I_syn_he, 'e')
    Phi_hi = Phi_I(I_syn_hi, 'i')
    Phi_hs= Phi_I(I_syn_hs, 'e')
    
    ## / Rate variable
    drE_dt[0][i] = (-r_E[0][i-1] + Phi_he) / taue
    drI_dt[0][i] = (-r_I[0][i-1] + Phi_hi) / taui
    drE_dt[1][i] = (-r_E[1][i-1] + Phi_hs) / taue

    dsaAmpa_dt = (- S_AMDA[0, i-1]/tauAMPA) + drE_dt[0][i] * dt + I_noise[0]
    dsGaba_dt = (- S_GABA[0, i-1]/tauGABA) + drI_dt[0][i] * dt + I_noise[1]
    dsNMDA_dt = (- S_NMDA[0, i-1] / tauNMDA) + gamma * drE_dt[1][i] *dt + I_noise[2]
    #"Not sure how the sum should be. hm is it dynamic ? or constant ?" hme or hmi ?
    dsmAmpa_dt = (- S_AMDA[1, i-1]/tauAMPA) + I_noise #+ np.sum(hm, 2)
    #have to ensure S>=0
    S_GABA[0,i] = max(0, S_GABA[0,i - 1]) + dsGaba_dt *dt  #random walk ??
    S_NMDA[0,i] = max(0,S_NMDA[0,i - 1]) + dsNMDA_dt *dt  #random walk


In [116]:
drI_dt

array([[  0.        , 101.52619235, 101.53010589, ..., 101.8301391 ,
        101.8301391 , 101.8301391 ]])

# Plot

We want to plot dr at the end for each population over the time